In [12]:
import time
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup



In [ ]:
# run in python:

import time
import numpy as np
import pandas as pd
import selenium.webdriver as sw

from bs4 import BeautifulSoup

HOST_WEB = 'https://carwiz.co.il'

driver = sw.Chrome(executable_path="./chromedriver.exe")
driver.get(HOST_WEB + '/used-cars')

v_columns = ['url','version' , 'year' , 'engine' , 'current_mileage' , 'hand' , 'gearBox' , 'color' , 'original_onership' , 'next_test_in' , 'annual_licensing_fee' , 'price']
v_indexes = [ 1 , 2, 3, 4, 5 , 6, 7, 8, 9 ,10 ]

car_urls = []
while (len(car_urls) < 300):
	
	driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
	
	time.sleep(1)
	
	try:
		soup = BeautifulSoup(driver.page_source, 'html.parser')
	
		elms = soup.select('#root div main div.MuiContainer-root.MuiContainer-maxWidthLg > div:nth-child(2) > div > div > div a[href]')
		
		#elms is a list of all  the links in the page , each link represents page for one car:
		i = 0
		for elm in elms:
			car_url = HOST_WEB + elm['href']
			if car_url not in car_urls:
				car_urls.append(car_url)
				print(car_url)
			print(len(car_urls))
	except:
		print('error')

df_car_urls=pd.DataFrame(data = np.array(car_urls))
df_car_urls.to_csv('cars_urls_data.csv')



In [143]:

Twelve_columns = ['version' , 'year' , 'engine' , 'current_mileage' , 'hand' , 'gearBox' , 'color' , 'original_onership' , 'next_test_in' , 'annual_licensing_fee' , "car_length" , 'price']

cars = pd.read_csv(r"C:\Users\avoav\JupyterProjects\project-semester-a\cars_urls_data.csv" ,index_col=0 )

df_Det = pd.DataFrame( columns= Twelve_columns)
counter = 0
for url in cars['url']:
    
    one_car = requests.get(url)
    soup = BeautifulSoup(one_car.content , 'html.parser')
     
   
    try:
        blocks = soup.find ('div',  xs = 'auto' )
        block = blocks.get_text()
        price = block.replace('\u200f', '').replace('\xa0₪', '').replace(',', '').replace('הצעת הסוכנותמחיר' , '')
        int(price)

    #if there is some urls which is not represented cars...
    except:
        print ("An exception occurred")
        continue

    #the length:     
    blocks1 = soup.select ('p' , tabindex_="0" )
    counter = 1
    for block in blocks1 :
        if (block.text == "אורך (ס''מ)"):
            length = blocks1[counter].text
        counter = counter+1
    
    Det = soup.select('td' , class_='MuiTableCell-root MuiTableCell-body MuiTableCell-alignLeft MuiTableCell-paddingNone')
        
    det_array = []

# choose only the second td each time?
    
    for i in range(len(Det)):
        if (i%2 != 0):
            det_array.append(Det[i].get_text())

    det_array.append(length)
    det_array.append(int(price)) 
    
#we choose only the cars which has all parameters:

    if (len(det_array) ==12):
        df_det_array = pd.DataFrame(np.array([det_array]) , columns = Twelve_columns  ) 
    
    #how to append data with different number of columns 
    df_Det = pd.concat([df_Det ,df_det_array] , ignore_index = True , sort = False).fillna(0)
    counter = counter +1
    if (counter % 300 == 0) : print(counter)

df_Det.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars_data.csv' , index_label='index' )


In [21]:
df_Det.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars_data.csv' , index_label='index' )
new_cars = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars_data.csv' , index_col  = 'index')

In [22]:
new_cars

,version,year,engine,current_mileage,hand,gearBox,color,original_onership,next_test_in,annual_licensing_fee,car_length,price
index,,,,,,,,,,,,
0,קופרה פורמנטור SILVER STORM,2021,1498 ליטר בנזין,"10,000",1,אוטומטי,לבן,פרטית,5 חודשים,"‏2,417 ₪",445.0,176000
1,טויוטה קורולה GLI,2011,1598 ליטר בנזין,"71,000",2,אוטומטי,כסף,פרטית,4 חודשים,"‏1,262 ₪",454.5,45000
2,טויוטה קורולה GLI,2011,1598 ליטר בנזין,"71,000",2,אוטומטי,כסף,פרטית,4 חודשים,"‏1,262 ₪",454.5,45000
3,קיה פיקנטו LX,2020,1248 ליטר בנזין,"16,500",2,אוטומטי,לבן,פרטית,9 חודשים,"‏1,235 ₪",359.5,79900
4,הונדה סיוויק COMFORT,2014,1798 ליטר בנזין,"144,000",2,אוטומטי,אפור,חברה,2 חודשים,"‏1,424 ₪",430.0,44900
...,...,...,...,...,...,...,...,...,...,...,...,...
1049,לנד רובר רינג' רובר ספורט PHEV AB DYN,2020,1997 ליטר חשמל/בנזין,"45,000",2,אוטומטי,לבן,פרטית,1 חודשים,"‏4,731 ₪",487.9,590000
1050,פיאט פנדה DYNAMIC,2011,1242 ליטר בנזין,"84,000",2,ידני,צהוב,פרטית,1 חודשים,‏880 ₪,353.8,13000
1051,ג'יפ צ'ירוקי LONGITUDE,2015,3239 ליטר בנזין,"180,000",4,אוטומטי,לבן,פרטית,6 חודשים,"‏1,962 ₪",462.4,79999


In [32]:
df_Det.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\cars_data.csv' , index_label='index' )

In [15]:
cars = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\cars_data.csv' , index_col  = 'index')

In [64]:
cars_f4719 = pd.read_csv(r"C:\Users\avoav\JupyterProjects\project-semester-a\cars_urls_data.csv" ,index_col=0 )
cars_f4719 = cars_f4719.iloc[4719:,:]
cars_f4719

df_Det = pd.DataFrame( columns= Twelve_columns)
counter = 0
for url in cars_f4719['url']:
    
    one_car = requests.get(url)
    soup = BeautifulSoup(one_car.content , 'html.parser')
     
   
    try:
        blocks = soup.find ('div',  xs = 'auto' )
        block = blocks.get_text()
        price = block.replace('\u200f', '').replace('\xa0₪', '').replace(',', '').replace('הצעת הסוכנותמחיר' , '')
        int(price)

    #if there is some urls which is not represented cars...
    except:
        print ("An exception occurred")
        continue

    #the length:     
    blocks1 = soup.select ('p' , tabindex_="0" )
    counter = 1
    for block in blocks1 :
        if (block.text == "אורך (ס''מ)"):
            length = blocks1[counter].text
        counter = counter+1
    
    Det = soup.select('td' , class_='MuiTableCell-root MuiTableCell-body MuiTableCell-alignLeft MuiTableCell-paddingNone')
        
    det_array = []
# how to choose only the second td each time?
    
    for i in range(len(Det)):
        if (i%2 != 0):
            det_array.append(Det[i].get_text())

    det_array.append(length)
    det_array.append(int(price)) 
    
#we choose only the cars which has all parameters:

    if (len(det_array) ==12):
        df_det_array = pd.DataFrame(np.array([det_array]) , columns = Twelve_columns  ) 
    
    #how to append data with different number of columns 
    df_Det = pd.concat([df_Det ,df_det_array] , ignore_index = True , sort = False).fillna(0)
    counter = counter +1
    if (counter % 300 == 0) : print(counter)

df_Det.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new1_cars_data.csv' , index_label='index' )


An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred


In [69]:
new_cars.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars2_data.csv' , index_label  = 'index')

In [65]:
df_Det.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars1_data.csv' , index_label  = 'index')
new_cars1 = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars1_data.csv' , index_col  = 'index')
new_cars = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars_data.csv' , index_col  = 'index')
new_cars = pd.concat([new_cars, new_cars1] ,  ignore_index = True  )

new_cars.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars_data.csv' , index_label  = 'index')


In [72]:
new_cars = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars_data.csv' , index_col  = 'index')

new_cars = new_cars.drop_duplicates()

new_cars.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars_data.csv' , index_label  = 'index')

In [16]:
cars.info()
#success just 965 urls so we will return the process from url number 1000. and 2000.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 0 to 2200
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   version               2201 non-null   object
 1   year                  2201 non-null   int64 
 2   CompanyName           2201 non-null   object
 3   engine                2201 non-null   object
 4   current_mileage       2201 non-null   object
 5   hand                  2201 non-null   int64 
 6   gearBox               2201 non-null   int64 
 7   color                 2201 non-null   object
 8   original_onership     2201 non-null   object
 9   next_test_in          2201 non-null   int64 
 10  annual_licensing_fee  2201 non-null   int64 
 11  price                 2201 non-null   int64 
dtypes: int64(6), object(6)
memory usage: 223.5+ KB


In [27]:
cars_f1000 = pd.read_csv(r"C:\Users\avoav\JupyterProjects\project-semester-a\cars_urls_data.csv" ,index_col=0 )
cars_f1000 = cars_f1000.iloc[1001:,:]
cars_f1000

,url
1001,https://carwiz.co.il/used-cars/04b86adc-b3c0-4...
1002,https://carwiz.co.il/used-cars/ce2a4faa-edab-4...
1003,https://carwiz.co.il/used-cars/216d5923-7b52-4...
1004,https://carwiz.co.il/used-cars/5244efa2-61ae-4...
1005,https://carwiz.co.il/used-cars/46f1ad87-2459-4...
...,...
3004,https://carwiz.co.il/used-cars/c47779ca-0b0c-4...
3005,https://carwiz.co.il/used-cars/997a0799-3db4-4...
3006,https://carwiz.co.il/used-cars/fdb72a7a-55a4-4...
3007,https://carwiz.co.il/used-cars/4a9e2a74-7b9c-4...


In [147]:
df_Det = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\cars_data.csv' , index_col  = 'index')
counter = 0
for url in cars_f1000['url']:
    
    one_car = requests.get(url)
    soup = BeautifulSoup(one_car.content , 'html.parser')
     
   
    try:
        blocks = soup.find ('div',  xs = 'auto' )
        block = blocks.get_text()
        price = block.replace('\u200f', '').replace('\xa0₪', '').replace(',', '').replace('הצעת הסוכנותמחיר' , '')
        int(price)

    #if there is some urls which is not represented cars...
    except:
        print ("An exception occurred")
        continue
    
    Det = soup.select('td' , class_='MuiTableCell-root MuiTableCell-body MuiTableCell-alignLeft MuiTableCell-paddingNone')
        
    det_array = []
# how to choose only the second td each time?
    
    for i in range(len(Det)):
        if (i%2 != 0):
            det_array.append(Det[i].get_text())
    
    det_array.append(int(price)) 
    
#we choose only the cars which has all parameters:

    if (len(det_array) ==11):
        df_det_array = pd.DataFrame(np.array([det_array]) , columns = eleven_columns  ) 
    
    #how to append data with different number of columns 
    df_Det = pd.concat([df_Det ,df_det_array] , ignore_index = True , sort = False).fillna(0)
    counter = counter +1
    if (counter % 300 == 0) : print(counter)

df_Det.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\cars_data.csv' , index_label='index' )


In [51]:
cars_f2000 = pd.read_csv(r"C:\Users\avoav\JupyterProjects\project-semester-a\cars_urls_data.csv" ,index_col=0 )
cars_f2000 = cars_f2000.iloc[2001:,:]
cars_f2000

,url
2001,https://carwiz.co.il/used-cars/ab01cd1c-16f5-4...
2002,https://carwiz.co.il/used-cars/265f1094-4a00-4...
2003,https://carwiz.co.il/used-cars/e05b494a-5a3e-4...
2004,https://carwiz.co.il/used-cars/b46d8adc-2cd3-4...
2005,https://carwiz.co.il/used-cars/4fb7b060-9899-4...
...,...
3004,https://carwiz.co.il/used-cars/c47779ca-0b0c-4...
3005,https://carwiz.co.il/used-cars/997a0799-3db4-4...
3006,https://carwiz.co.il/used-cars/fdb72a7a-55a4-4...
3007,https://carwiz.co.il/used-cars/4a9e2a74-7b9c-4...


In [144]:
df_Det = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\cars_data.csv' , index_col  = 'index')
counter = 0
for url in cars_f2000['url']:
    
    one_car = requests.get(url)
    soup = BeautifulSoup(one_car.content , 'html.parser')
     
   
    try:
        blocks = soup.find ('div',  xs = 'auto' )
        block = blocks.get_text()
        price = block.replace('\u200f', '').replace('\xa0₪', '').replace(',', '').replace('הצעת הסוכנותמחיר' , '')
        int(price)

    #if there is some urls which is not represented cars...
    except:
        print ("An exception occurred")
        continue
    
    Det = soup.select('td' , class_='MuiTableCell-root MuiTableCell-body MuiTableCell-alignLeft MuiTableCell-paddingNone')
        
    det_array = []
# how to choose only the second td each time?
    
    for i in range(len(Det)):
        if (i%2 != 0):
            det_array.append(Det[i].get_text())
    
    det_array.append(int(price)) 
    
#we choose only the cars which has all parameters:

    if (len(det_array) ==11):
        df_det_array = pd.DataFrame(np.array([det_array]) , columns = eleven_columns  ) 
    
    #how to append data with different number of columns 
    df_Det = pd.concat([df_Det ,df_det_array] , ignore_index = True , sort = False).fillna(0)
    counter = counter +1
    if (counter % 300 == 0) : print(counter)

df_Det.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\cars_data.csv' , index_label='index' )


In [46]:
df_Det.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\cars_data.csv' , index_label='index' )

In [73]:
cars = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars_data.csv' , index_col  = 'index')

In [74]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3002 entries, 0 to 4905
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   version               3002 non-null   object 
 1   year                  3002 non-null   int64  
 2   engine                3002 non-null   object 
 3   current_mileage       3002 non-null   object 
 4   hand                  3002 non-null   int64  
 5   gearBox               3002 non-null   object 
 6   color                 3002 non-null   object 
 7   original_onership     3002 non-null   object 
 8   next_test_in          3002 non-null   object 
 9   annual_licensing_fee  3002 non-null   object 
 10  car_length            3002 non-null   float64
 11  price                 3002 non-null   int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 304.9+ KB


In [75]:
#we have got 2057 cars but just 573 different types. so we need to try to get more...
len(cars.version.unique())

928

In [85]:
cars.version.unique()
#we saw here that we need to split the "version" column to two columns : "CompanyName" and "version":
CompanyName = cars['version'].apply(lambda x : x.split(' ')[0])
cars.insert(1,"CompanyName",CompanyName)
cars

,version,CompanyName,year,engine,current_mileage,hand,gearBox,color,original_onership,next_test_in,annual_licensing_fee,car_length,price
0,קופרה פורמנטור SILVER STORM,קופרה,2021,1498 ליטר בנזין,"10,000",1,אוטומטי,לבן,פרטית,5 חודשים,"‏2,417 ₪",445.0,176000
1,טויוטה קורולה GLI,טויוטה,2011,1598 ליטר בנזין,"71,000",2,אוטומטי,כסף,פרטית,4 חודשים,"‏1,262 ₪",454.5,45000
2,קיה פיקנטו LX,קיה,2020,1248 ליטר בנזין,"16,500",2,אוטומטי,לבן,פרטית,9 חודשים,"‏1,235 ₪",359.5,79900
3,הונדה סיוויק COMFORT,הונדה,2014,1798 ליטר בנזין,"144,000",2,אוטומטי,אפור,חברה,2 חודשים,"‏1,424 ₪",430.0,44900
4,פיאט 500 1.2 POP STAR,פיאט,2017,1242 ליטר בנזין,"69,000",1,אוטומטי,לבן,פרטית,2 חודשים,"‏1,353 ₪",357.1,42900
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2997,סקודה קודיאק AMBITION,סקודה,2017,1395 ליטר בנזין,"159,800",1,אוטומטי,זהב,ליסינג,5 חודשים,"‏2,019 ₪",448.0,132000
2998,סקודה פאביה AMBITION,סקודה,2016,1197 ליטר בנזין,"140,000",3,אוטומטי,שחור מטלי,ליסינג,4 חודשים,"‏1,102 ₪",399.2,59000
2999,מאזדה 3,מאזדה,2020,1998 ליטר בנזין,"14,500",1,אוטומטי,אפור,פרטית,11 חודשים,"‏2,130 ₪",466.0,130500
3000,קרייזלר PT-CRUISE LIMITED,קרייזלר,2008,2429 ליטר בנזין,"163,000",3,אוטומטי,אחר,פרטית,3 חודשים,"‏1,427 ₪",429.0,17000


In [145]:
cars = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\new_cars_data.csv' , index_col  = 'index')
cars = cars.reset_index(drop=True)
cars

,version,year,engine,current_mileage,hand,gearBox,color,original_onership,next_test_in,annual_licensing_fee,car_length,price
0,קופרה פורמנטור SILVER STORM,2021,1498 ליטר בנזין,"10,000",1,אוטומטי,לבן,פרטית,5 חודשים,"‏2,417 ₪",445.0,176000
1,טויוטה קורולה GLI,2011,1598 ליטר בנזין,"71,000",2,אוטומטי,כסף,פרטית,4 חודשים,"‏1,262 ₪",454.5,45000
2,קיה פיקנטו LX,2020,1248 ליטר בנזין,"16,500",2,אוטומטי,לבן,פרטית,9 חודשים,"‏1,235 ₪",359.5,79900
3,הונדה סיוויק COMFORT,2014,1798 ליטר בנזין,"144,000",2,אוטומטי,אפור,חברה,2 חודשים,"‏1,424 ₪",430.0,44900
4,פיאט 500 1.2 POP STAR,2017,1242 ליטר בנזין,"69,000",1,אוטומטי,לבן,פרטית,2 חודשים,"‏1,353 ₪",357.1,42900
...,...,...,...,...,...,...,...,...,...,...,...,...
2997,סקודה קודיאק AMBITION,2017,1395 ליטר בנזין,"159,800",1,אוטומטי,זהב,ליסינג,5 חודשים,"‏2,019 ₪",448.0,132000
2998,סקודה פאביה AMBITION,2016,1197 ליטר בנזין,"140,000",3,אוטומטי,שחור מטלי,ליסינג,4 חודשים,"‏1,102 ₪",399.2,59000
2999,מאזדה 3,2020,1998 ליטר בנזין,"14,500",1,אוטומטי,אפור,פרטית,11 חודשים,"‏2,130 ₪",466.0,130500
3000,קרייזלר PT-CRUISE LIMITED,2008,2429 ליטר בנזין,"163,000",3,אוטומטי,אחר,פרטית,3 חודשים,"‏1,427 ₪",429.0,17000


In [93]:
#add more data:
big_cars_d = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\big_cars_data.csv' , index_col  = 'index')
cars = cars.concat([cars ,big_cars_d] , ignore_index = True , sort = False).fillna(0)

In [94]:
cars.to_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\cars_data.csv' , index_label  = 'index')